<a href="https://colab.research.google.com/github/NLP-trade-performance-predictor/QQQ-stock-prediction/blob/master/QQQ_stock_prediction_headlines_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Init Notebook


*   mount drive
*   install `transformers`
*   import packages
*   import `BERT`



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qq transformers

     |████████████████████████████████| 2.2MB 20.1MB/s 
     |████████████████████████████████| 3.3MB 38.3MB/s 
     |████████████████████████████████| 870kB 48.5MB/s 


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

import pandas as pd
import datetime as dt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
model = BertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# Import Dataset



*   import `articles` and `stocks` csv files
*   join them into one `pandas` dataframe by `date`



In [ ]:
PATH='/content/drive/MyDrive/NLP Final Project'

In [ ]:
stocks = pd.read_csv(f'{PATH}/stocks.csv', header=None)
articles = pd.read_csv(f'{PATH}/articles.csv', header=None)

In [ ]:
stocks.head(10)

,0,1,2,3
0,2003-10-21,0.11,0.003130,1
1,2016-07-07,0.23,0.002122,1
2,2012-03-19,0.37,0.005554,2
3,2020-07-02,-0.83,-0.003280,1
4,2003-11-24,0.66,0.019119,2
5,2014-10-16,1.36,0.015039,2
6,2008-11-20,-0.98,-0.036925,0
7,2001-06-27,0.11,0.002518,1
8,2014-02-24,0.31,0.003441,1
9,2003-09-02,0.38,0.011347,2


In [ ]:
articles.head(10)

,0,1,2
0,Facebook and Google reveal plans to build subs...,Mon 29 Mar 2021 12:34:00 GMT,CNBC
1,What Does a Google Partnership Mean for Wix? -...,Sun 28 Mar 2021 13:00:00 GMT,The Motley Fool
2,Classic Hangouts to Google Chat transition: Wh...,Mon 29 Mar 2021 14:00:00 GMT,TechRepublic
3,Google: No Benefit to An Artificially Flat URL...,Sun 28 Mar 2021 18:25:50 GMT,Search Engine Journal
4,Mtn. View signs off on Google's massive plan f...,Sun 28 Mar 2021 03:50:14 GMT,Los Altos Town Crier
5,Google Aims to Be the Anti-Amazon of E-Commerc...,Mon 29 Mar 2021 01:42:00 GMT,The New York Times
6,Google's App Store Privacy Labels: What They M...,Sun 28 Mar 2021 04:22:26 GMT,Gadgets 360
7,Ex-Google Policy Lead Founds Tech-Friendly Pro...,Mon 29 Mar 2021 12:04:28 GMT,Yahoo Finance
8,Google Facebook Twitter grilled in US on fake ...,Thu 25 Mar 2021 20:07:06 GMT,BBC News
9,Google offered a professor $60000 but he turne...,Wed 24 Mar 2021 21:46:00 GMT,CNN


In [ ]:
articles[1] = articles[1].map(lambda article: dt.datetime.strftime(dt.datetime.strptime(article, '%a %d %b %Y %H:%M:%S GMT'), '%Y-%m-%d'))

In [ ]:
joined_df = articles.set_index(1).join(stocks.set_index(0), lsuffix='article', rsuffix='stock')
joined_df = joined_df.dropna()
joined_df = joined_df.rename({0: 'headline', '2article': 'publisher', 1: 'diff', '2stock': 'ratio', 3: 'label'}, axis=1)
joined_df['headline'] = joined_df['headline'].map(lambda headline: headline[:headline.rindex(' - ')])
joined_df['label'] = joined_df['label'].astype(int)
joined_df.head(10)

,headline,publisher,diff,ratio,label
2010-01-04,Windows Mobile Bug Dates Messages from 2016,Wired News,0.1,0.002159,1
2010-01-04,Understanding Windows 7's 'GodMode',CNET News,0.1,0.002159,1
2010-01-04,NetworkWorld: Facebook blocks 'Web 2.0 Suicide...,NetworkWorld.com,0.1,0.002159,1
2010-01-04,Social Media - Top Facebook Status Trends of 2009,MarketingProfs.com (subscription),0.1,0.002159,1
2010-01-04,Worlds Collide: Twitter Has More Uptime Than F...,TechCrunch,0.1,0.002159,1
2010-01-04,Report: Apple scheduling Jan. 27 media event,Macworld,0.1,0.002159,1
2010-01-04,Apple Product Announcement Scheduled in San Fr...,All Things Digital,0.1,0.002159,1
2010-01-04,Report: Wednesday Jan. 27 Set For Major Apple ...,Cult of Mac,0.1,0.002159,1
2010-01-04,"Japanese Apple Store shoppers get ""Lucky Bags""...",Cult of Mac,0.1,0.002159,1
2010-01-04,Isaac Newton: Why apples are falling on Google...,Christian Science Monitor,0.1,0.002159,1


# BERT Model

In [ ]:
df_train, df_test = train_test_split(
  joined_df,
  test_size=0.2,
)
df_train, df_val = train_test_split(
  df_train,
  test_size=0.15,
)

In [ ]:
train_encodings = tokenizer(df_train['headline'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(df_val['headline'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(df_test['headline'].tolist(), truncation=True, padding=True)

In [ ]:
class HeadlinesStocksDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = HeadlinesStocksDataset(train_encodings, df_train['label'].tolist())
val_dataset = HeadlinesStocksDataset(val_encodings, df_val['label'].tolist())
test_dataset = HeadlinesStocksDataset(test_encodings, df_test['label'].tolist())

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

Step,Training Loss
10,1.171600
20,1.194500
30,1.134800
40,1.119500
50,1.092100
60,1.079300
70,1.086000
80,1.100800
90,1.112100
100,1.099700


TrainOutput(global_step=510, training_loss=1.074081080567603, metrics={'train_runtime': 151.6626, 'train_samples_per_second': 3.363, 'total_flos': 379989988153488.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2367795200, 'init_mem_gpu_alloc_delta': 433779200, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 13611008, 'train_mem_gpu_alloc_delta': 1328218624, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 893712384})

In [ ]:
trainer.evaluate(test_dataset)

{'epoch': 3.0,
 'eval_loss': 1.0923235416412354,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 76417024,
 'eval_runtime': 1.6029,
 'eval_samples_per_second': 497.21}

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

correct_predictions = 0

with torch.no_grad():
  for data in test_loader:
      input_ids = data['input_ids'].to(device)
      attention_mask = data['attention_mask'].to(device)
      labels = data['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask).logits.squeeze()
      # for accuracy calculation
      y_pred_softmax = torch.log_softmax(outputs, dim = 1)
      _, predicted_labels = torch.max(y_pred_softmax, dim = 1)    

      correct_predictions += torch.sum(predicted_labels == labels)

print(correct_predictions.item() / len(test_dataset))

0.40401505646173147
